[Electric Vehicle Population Data](https://catalog.data.gov/dataset/electric-vehicle-population-data)<br>
Metadata Updated: July 20, 2024
<br><br>
This dataset shows the Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through Washington State Department of Licensing (DOL).

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("all-MiniLM-L6-v2")
from sklearn.metrics.pairwise import cosine_similarity
import copy
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq
import requests

data = pd.read_csv("../data/data.gov/Electric_Vehicle_Population_Data.csv")
data["CAFV_indicator"] = data["Clean Alternative Fuel Vehicle (CAFV) Eligibility"] == "Clean Alternative Fuel Vehicle Eligible"
data["Make-Model-Year"] = data[["Make", "Model", "Model Year"]].apply(lambda g: g[0] + "-" + g[1] + "-" + str(g[2]), axis=1)
data["Model-Year"] = data[["Model", "Model Year"]].apply(lambda g: g[0]+"-"+str(g[1]),axis=1)

exempt_list = pd.read_csv("../data/data.gov/WA_Tax_Exemptions_-_Potential_Eligibility_by_Make_Model_Excluding_Vehicle_Price_Criteria_20240730.csv")

In [2]:
data.columns.tolist()

['VIN (1-10)',
 'County',
 'City',
 'State',
 'Postal Code',
 'Model Year',
 'Make',
 'Model',
 'Electric Vehicle Type',
 'Clean Alternative Fuel Vehicle (CAFV) Eligibility',
 'Electric Range',
 'Base MSRP',
 'Legislative District',
 'DOL Vehicle ID',
 'Vehicle Location',
 'Electric Utility',
 '2020 Census Tract',
 'CAFV_indicator',
 'Make-Model-Year',
 'Model-Year']

"To be eligible for the exemptions, the vehicle must be capable of traveling at least thirty miles using only battery power."

In [3]:
data["Clean Alternative Fuel Vehicle (CAFV) Eligibility"].value_counts(normalize=True, dropna=False) 


Eligibility unknown as battery range has not been researched    0.538717
Clean Alternative Fuel Vehicle Eligible                         0.354298
Not eligible due to low battery range                           0.106985
Name: Clean Alternative Fuel Vehicle (CAFV) Eligibility, dtype: float64

In [4]:
data.loc[data.CAFV_indicator]["Electric Range"].value_counts(dropna=False).sort_index()

30.0     1636
31.0       97
32.0     4607
33.0     1346
34.0      738
         ... 
308.0     508
322.0    1717
330.0     331
337.0      82
NaN         2
Name: Electric Range, Length: 81, dtype: int64

In [5]:
data["Electric Vehicle Type"].value_counts()

Battery Electric Vehicle (BEV)            152179
Plug-in Hybrid Electric Vehicle (PHEV)     42053
Name: Electric Vehicle Type, dtype: int64

Note from the 'About this Dataset' section: <br>
"Electric Range is no longer maintained for Battery Electric Vehicles (BEV) because new BEVs have an electric range of 30 miles or more. Zero (0) will be entered where the electric range has not been researched."

It might be that "new BEVs" from the note is those made in year starting from 2021

In [6]:
notCAFV = data.loc[~data.CAFV_indicator & (data["Electric Range"] == 0)]
notCAFV = copy.deepcopy(notCAFV)
notCAFV.loc[notCAFV["Electric Vehicle Type"].map(lambda x: x.endswith("(BEV)")), "Model Year"].value_counts(dropna=False).sort_index(ascending=False)

2025       18
2024    13793
2023    52165
2022    23730
2021    14839
2020       82
2019        3
2013        6
Name: Model Year, dtype: int64

Creating similarity score with the tax exemption dataset of vehicle model description imported from the website.<br>
This is to see if the newer BEV are eligible for CAFV criteria

In [7]:

similarity_list = []
for data_modelname in set(data.loc[(data["Electric Vehicle Type"].map(lambda x: x.endswith("(BEV)")))&(data["Electric Range"] == 0)]["Model"].tolist()):
    for exempt_modelname in set(exempt_list["Vehicle Model Description"].tolist()):
        sim_score = fuzz.ratio(data_modelname.lower(), exempt_modelname.lower())
        # sim_score = cosine_similarity(embeddings1, embeddings2)[0][0]
        # embeddings1 = model.encode([data_modelname], normalize_embeddings=True)
        # embeddings2 = model.encode([exempt_modelname], normalize_embeddings=True)
        similarity_list.append((data_modelname, exempt_modelname, sim_score))
        # print(f"{data_modelname} and {exempt_modelname} have {sim_score}")
similarity_list=sorted(similarity_list, key=lambda x: x[-1], reverse=True)

similarity_dict = {}
for model_name, exempt_name, score in similarity_list:
    # print(model_name, exempt_name, score)
    if model_name not in similarity_dict:
        similarity_dict[model_name] = exempt_name, score
        
notCAFV['exempt_list_name'] = None
notCAFV['sim_score'] = None
notCAFV["exempt_list_name"] = notCAFV["Model"].map(lambda x: similarity_dict[x][0])
notCAFV["sim_score"] = notCAFV["Model"].map(lambda x: similarity_dict[x][1])

Using the similarity score, many of the vehicle models are in the exemption list made after year 2020.
- For year 2025, it's in the future and I think it is safe to assume the future BEVs are already CAFV without having to know electric mileage based on the note from the website.
- Thus, I will assign CAFV indicator as `True` with BEV model year on and after 2020 that have electric mileage as zero

In [8]:
notCAFV.groupby("Model Year").agg({"sim_score":"mean"}).sort_index()

,sim_score
Model Year,
2013,57.000000
2019,71.000000
2020,89.939024
2021,88.365321
2022,85.621955
2023,85.063357
2024,79.247734
2025,71.111111


In [9]:
data.iloc[notCAFV.loc[notCAFV["Model Year"] >= 2020].index.tolist(), 17] = True
# data.to_csv("dataCAFV.csv", index=False)

In [139]:
data.groupby("Electric Vehicle Type").agg({"CAFV_indicator":"mean"})

,CAFV_indicator
Electric Vehicle Type,
Battery Electric Vehicle (BEV),0.999888
Plug-in Hybrid Electric Vehicle (PHEV),0.506052


In [58]:
data.loc[(data["Electric Vehicle Type"].map(lambda x: x.endswith("(PHEV)"))) & (data['Electric Range'] > 0)].CAFV_indicator.mean()

0.5060283940928872

In [141]:
data['Make'].value_counts(normalize=True, dropna=False)

TESLA                   0.439269
CHEVROLET               0.075055
NISSAN                  0.073932
FORD                    0.052854
KIA                     0.043747
BMW                     0.041960
TOYOTA                  0.037955
VOLKSWAGEN              0.028229
JEEP                    0.026468
HYUNDAI                 0.026334
RIVIAN                  0.025289
VOLVO                   0.023508
AUDI                    0.020527
CHRYSLER                0.017937
MERCEDES-BENZ           0.009437
PORSCHE                 0.006225
SUBARU                  0.005514
POLESTAR                0.005349
MITSUBISHI              0.005169
MINI                    0.005143
HONDA                   0.004541
FIAT                    0.004083
LEXUS                   0.003300
DODGE                   0.003249
CADILLAC                0.003244
MAZDA                   0.003244
LINCOLN                 0.001483
LUCID                   0.001364
SMART                   0.001333
GENESIS                 0.001230
JAGUAR    

In [17]:
data["Model"].value_counts(normalize=True, dropna=False)

MODEL Y           0.206809
MODEL 3           0.159320
LEAF              0.069510
MODEL S           0.040266
BOLT EV           0.033661
                    ...   
SPECTRE           0.000010
PROMASTER 3500    0.000010
S-10 PICKUP       0.000005
918               0.000005
ESPRINTER         0.000005
Name: Model, Length: 151, dtype: float64

In [28]:
phev = copy.deepcopy(data.loc[data["Electric Vehicle Type"].map(lambda x: x.endswith("(PHEV)"))])
phev.groupby("Make").agg({"CAFV_indicator": "mean", "Model": "count"}).\
rename(columns={"CAFV_indicator": "proportion of CAFV", "Model": "count"}).\
sort_values(["proportion of CAFV", "count"], ascending=False)

,proportion of CAFV,count
Make,,
CHEVROLET,1.000000,4833
CHRYSLER,1.000000,3484
DODGE,1.000000,631
LEXUS,1.000000,347
CADILLAC,1.000000,88
ALFA ROMEO,1.000000,59
FISKER,1.000000,14
WHEEGO ELECTRIC CARS,1.000000,3
HONDA,0.988152,844


In [29]:
phev_partial=\
[
'HONDA',
'KIA',
'HYUNDAI',
'BMW',
'VOLVO',
'TOYOTA',
'MITSUBISHI',
'MERCEDES-BENZ',
'LAND ROVER',
'FORD',
'PORSCHE'
]

In [30]:
phev.loc[phev["Make"].isin(phev_partial)]

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract,CAFV_indicator,Make-Model-Year,Model-Year
4,5UXTA6C09P,Snohomish,Monroe,WA,98272.0,2023,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30.0,0.0,39.0,235249262,POINT (-121.968385 47.854897),PUGET SOUND ENERGY INC,5.306105e+10,True,BMW-X5-2023,X5-2023
5,1FMCU0EZXN,Yakima,Moxee,WA,98936.0,2022,FORD,ESCAPE,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38.0,0.0,15.0,197264322,POINT (-120.3552229 46.550869),PACIFICORP,5.307700e+10,True,FORD-ESCAPE-2022,ESCAPE-2022
8,WP1AE2A21J,Snohomish,Everett,WA,98208.0,2018,PORSCHE,CAYENNE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14.0,0.0,44.0,122759276,POINT (-122.2032349 47.8956271),PUGET SOUND ENERGY INC,5.306104e+10,False,PORSCHE-CAYENNE-2018,CAYENNE-2018
14,5UXTS1C01M,Thurston,Rochester,WA,98579.0,2021,BMW,X3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,17.0,0.0,20.0,152190328,POINT (-123.0498592 46.8153419),PUGET SOUND ENERGY INC,5.306701e+10,False,BMW-X3-2021,X3-2021
18,5UXTS1C06M,King,Seattle,WA,98199.0,2021,BMW,X3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,17.0,0.0,36.0,132733793,POINT (-122.394937 47.649547),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10,False,BMW-X3-2021,X3-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194183,KNDCC3LD3J,Pierce,Tacoma,WA,98403.0,2018,KIA,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26.0,0.0,27.0,267979911,POINT (-122.458462 47.264455),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,5.305306e+10,False,KIA-NIRO-2018,NIRO-2018
194185,KNDCD3LD0K,Pierce,Tacoma,WA,98408.0,2019,KIA,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26.0,0.0,27.0,243621605,POINT (-122.4445098 47.2002928),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,5.305306e+10,False,KIA-NIRO-2019,NIRO-2019
194189,JTDACACU6P,Benton,Kennewick,WA,99337.0,2023,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,39.0,0.0,8.0,257657494,POINT (-119.106637 46.173866),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF B...,5.300501e+10,True,TOYOTA-PRIUS PRIME-2023,PRIUS PRIME-2023
194207,JTDKN3DP8C,Jefferson,Port Townsend,WA,98368.0,2012,TOYOTA,PRIUS PLUG-IN,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,6.0,0.0,24.0,317207334,POINT (-122.7862046 48.1129794),BONNEVILLE POWER ADMINISTRATION||PUGET SOUND E...,5.303195e+10,False,TOYOTA-PRIUS PLUG-IN-2012,PRIUS PLUG-IN-2012
